### Preprocessing dataset metadata

In [1]:
from preprocess import *

# preprocess and load txt ds metadata
ds_meta = build_ds_meta()

### Dataset analysis

In [2]:
for ds_meta_item in ds_meta:
    print('Quantity of %s: %s' % (ds_meta_item, ds_meta[ds_meta_item].count()[0]))

Quantity of bounding_boxes: 48562
Quantity of classes: 1011
Quantity of hierarchy: 1010
Quantity of image_class_labels: 48562
Quantity of images: 48562
Quantity of photographers: 48562
Quantity of sizes: 48562


![](assets/dataset.png)

### Preprocessing dataset

In [ ]:
# BASE               SET_A
# + Bounding box  => SET_A_BB               (del)
# + Gabor filter  => SET_A_BB_GF            (del)
# + TVT splitting => SET_A_BB_GF_train
#                    SET_A_BB_GF_validation
#                    SET_A_BB_GF_test

apply_bounding_box('data/SET_A', 'data/SET_A_BB', ds_meta)
apply_gabor_filter('data/SET_A_BB', 'data/SET_A_BB_GF')
apply_tvt_split('data/SET_A_BB_GF')

### Building simple perceptron with Keras

In [181]:
import os

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense

In [182]:
batch_size = 16

train_labels = os.listdir('data/SET_A_BB_GF_train')

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    'data/SET_A_BB_GF_train',
    target_size=(150, 150),
    batch_size=batch_size,
    classes=train_labels)

model = Sequential()
model.add(Flatten(input_shape=(150, 150, 3))) 
model.add(Dense(64))
model.add(Dense(64))
model.add(Dense(len(train_labels)))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50)

Found 2103 images belonging to 50 classes.
Epoch 1/50
125/125 [==============================] - 36s 287ms/step - loss: 8.5559 - acc: 0.0210
Epoch 2/50
125/125 [==============================] - 38s 304ms/step - loss: 8.2159 - acc: 0.0195
Epoch 3/50
125/125 [==============================] - 38s 302ms/step - loss: 8.3293 - acc: 0.0200
Epoch 4/50
125/125 [==============================] - 37s 295ms/step - loss: 8.2132 - acc: 0.0216
Epoch 5/50
125/125 [==============================] - 39s 314ms/step - loss: 8.2265 - acc: 0.0170
Epoch 6/50
125/125 [==============================] - 39s 311ms/step - loss: 8.4052 - acc: 0.0220
Epoch 7/50
125/125 [==============================] - 37s 298ms/step - loss: 8.2629 - acc: 0.0175
Epoch 8/50
125/125 [==============================] - 41s 329ms/step - loss: 8.2005 - acc: 0.0205
Epoch 9/50
125/125 [==============================] - 40s 319ms/step - loss: 8.3174 - acc: 0.0195
Epoch 10/50
125/125 [==============================] - 37s 293ms/step - los

KeyboardInterrupt: 

### References

* https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
* BSIF http://www.ee.oulu.fi/~jkannala/bsif/bsif.pdf